In [1]:
import sys
from pathlib import Path

SRC_DIR = Path(r"C:\Users\manue\OneDrive\Escritorio\Proyecto WhoScored\src")
if str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

# Import correcto: el módulo está dentro del paquete whoscored_viz
from whoscored_viz import whoscored_matchcenter as wsm

print("Módulo cargado desde:", wsm.__file__)

[paths.py] PROJECT_ROOT: c:\Users\manue\OneDrive\Escritorio\Proyecto WhoScored
[paths.py] BASE_DATA_DIR: C:\Users\manue\OneDrive\Escritorio\Proyecto WhoScored\data
Módulo cargado desde: C:\Users\manue\OneDrive\Escritorio\Proyecto WhoScored\src\whoscored_viz\whoscored_matchcenter.py


In [2]:
from pathlib import Path
import re
import pandas as pd

# CSV de entrada que te pasa el notebook anterior
CSV_PATH = Path(r"C:\Users\manue\OneDrive\Escritorio\Proyecto WhoScored\data\raw\fixtures\DataFixtures\laliga\2025-2026\finished_matches.csv")

# OJO con esta ruta: el módulo escribe en out_root / "MatchCenter" / ...
# Para cuadrar con tu estructura existente (...\matchcenter\MatchCenter\...), pásale out_root = ...\matchcenter
OUT_ROOT = Path(r"C:\Users\manue\OneDrive\Escritorio\Proyecto WhoScored\data\raw\matchcenter")

# Carpeta donde YA tienes los partidos (la que termina en \MatchCenter)
EXISTING_MATCHCENTER_DIR = OUT_ROOT / "MatchCenter"

In [3]:
ID_FROM_URL = [
    re.compile(r"/(\d{5,})[/\-]?$"),          # .../1913916  o .../1913916/
    re.compile(r"[_-](\d{5,})$"),             # sufijo ..._1913916
    re.compile(r"[?&](?:matchId|gameId)=(\d+)")  # ?matchId=1913916
]

def extract_match_id_any(s: str) -> str | None:
    s = str(s or "").strip()
    if not s:
        return None
    if s.isdigit():
        return s
    for rx in ID_FROM_URL:
        m = rx.search(s)
        if m:
            return m.group(1)
    return None

def get_extracted_match_ids(root_matchcenter_dir: Path) -> set[str]:
    """
    Recorre recursivamente ...\MatchCenter\...\<carpeta del partido> y recoge IDs
    asumiendo que el nombre termina en _<ID>.
    """
    ids = set()
    for p in root_matchcenter_dir.rglob("*"):
        if p.is_dir():
            m = re.search(r"_(\d+)$", p.name)
            if m:
                ids.add(m.group(1))
    return ids

In [4]:
df_in = pd.read_csv(CSV_PATH)

# Intenta columna match_id directa; si no, extrae desde las URLs habituales
if "match_id" in df_in.columns:
    ids_input = df_in["match_id"].astype(str)
else:
    url_col = None
    for c in ("match_centre_url", "match_center_url", "match__centre_url", "match_url", "url"):
        if c in df_in.columns:
            url_col = c; break
    if not url_col:
        raise ValueError("El CSV debe incluir 'match_id' o alguna columna de URL (match_centre_url/match_center_url).")
    ids_input = df_in[url_col].map(extract_match_id_any)

# Limpia nulos
df_in["match_id_norm"] = ids_input
df_in = df_in[df_in["match_id_norm"].notna()].copy()
df_in["match_id_norm"] = df_in["match_id_norm"].astype(str)

# IDs ya presentes en disco
extracted_ids = get_extracted_match_ids(EXISTING_MATCHCENTER_DIR)

# Qué falta por descargar
df_pending = df_in[~df_in["match_id_norm"].isin(extracted_ids)].copy()

print(f"Total CSV: {len(df_in)}  |  Ya en disco: {len(extracted_ids)}  |  Pendientes: {len(df_pending)}")
if df_pending.empty:
    print("Nada que hacer. Todo descargado.")

Total CSV: 41  |  Ya en disco: 31  |  Pendientes: 10


In [6]:
from tempfile import NamedTemporaryFile
from pathlib import Path
from whoscored_viz import whoscored_matchcenter as wsm # tu módulo
from selenium import webdriver

if not df_pending.empty:
    # Crear CSV temporal con mismas columnas que el original pero solo filas pendientes
    tmp = NamedTemporaryFile(prefix="pending_", suffix=".csv", delete=False)
    tmp_path = Path(tmp.name)
    tmp.close()
    df_pending.to_csv(tmp_path, index=False, encoding="utf-8-sig")

    # Reusar una sola sesión de Selenium (más estable y rápido)
    driver = wsm._build_driver(headless=False)  # sí, es "privado", pero es tu módulo. Funciona. :contentReference[oaicite:0]{index=0}
    try:
        wsm.process_from_csv(
            csv_file=tmp_path,
            out_root=OUT_ROOT,        # importante: OUT_ROOT sin el último "MatchCenter"
            driver=driver,            # misma sesión para todas
            pause_range=(1.2, 2.8),
            cooldown_every=8,
            cooldown_secs=20,
            limit=None,               # procesa todas las pendientes
        )  # internamente usa process_one_match → save_all_tables con la estructura habitual. :contentReference[oaicite:1]{index=1}
    finally:
        driver.quit()
        tmp_path.unlink(missing_ok=True)

✅ OK [1/10] match_id=1913929 → C:\Users\manue\OneDrive\Escritorio\Proyecto WhoScored\data\raw\matchcenter\MatchCenter\Competition\Season\20250912_Sevilla_vs_Elche_1913929
✅ OK [2/10] match_id=1913925 → C:\Users\manue\OneDrive\Escritorio\Proyecto WhoScored\data\raw\matchcenter\MatchCenter\Competition\Season\20250913_Getafe_vs_Real_Oviedo_1913925
✅ OK [3/10] match_id=1913930 → C:\Users\manue\OneDrive\Escritorio\Proyecto WhoScored\data\raw\matchcenter\MatchCenter\Competition\Season\20250913_Real_Sociedad_vs_Real_Madrid_1913930
✅ OK [4/10] match_id=1913927 → C:\Users\manue\OneDrive\Escritorio\Proyecto WhoScored\data\raw\matchcenter\MatchCenter\Competition\Season\20250913_Athletic_Club_vs_Deportivo_Alaves_1913927
✅ OK [5/10] match_id=1913921 → C:\Users\manue\OneDrive\Escritorio\Proyecto WhoScored\data\raw\matchcenter\MatchCenter\Competition\Season\20250913_Atletico_vs_Villarreal_1913921
✅ OK [6/10] match_id=1913923 → C:\Users\manue\OneDrive\Escritorio\Proyecto WhoScored\data\raw\matchcenter